In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import os

In [97]:
# Sources:
# https://digiconomist.net/ethereum-energy-consumption/
# https://etherscan.io/chart/tx

print(f"=== [ Remember to download the relevant CSV files to {os.getcwd()} ! ] ===")

=== [ Remember to download the relevant CSV files to /home/spdskatr/Code/Hack-Cambridge-2022/scratch ! ] ===


# Part 0 - Getting the Data

In [92]:
# Data from Digiconomist

ETH_ENERGY = pd.read_csv("./ethereum-energy-consumpt.csv")

# Data from Etherscan (Attribution required)

ETH_DAILY_TRANSACTIONS = pd.read_csv("./export-TxGrowth.csv")

# Part 1 - Energy Consumption by Ethereum Blockchain Per Day

In [112]:
eth_energy_formatted = ETH_ENERGY \
    .astype({ "DateTime": np.datetime64 }) \
    .set_index("DateTime") \
    ["Estimated TWh per Year"]

eth_energy_formatted

DateTime
2017-05-20      2.338304
2017-05-21      2.331936
2017-05-22      2.406776
2017-05-23      2.479116
2017-05-24      2.558065
                 ...    
2022-01-17    107.791271
2022-01-18    108.195707
2022-01-19    108.388245
2022-01-20    108.577755
2022-01-21    108.746580
Name: Estimated TWh per Year, Length: 1703, dtype: float64

In [113]:
eth_energy_formatted.loc[datetime.datetime(2020, 5, 5)]

7.8016677108333345

In [131]:
# This function queries the number of TWh per year for any specific UNIX timestamp

def query_twh_per_yr(lookup, epoch):
    tab = lookup.loc[:np.datetime64(epoch, "s")]
    if not tab.empty:
        # Get last index you found
        return tab.iloc[-1]
    else:
        return lookup.iloc[0]

In [132]:
query_twh_per_yr(eth_energy_formatted, 1641837499)

106.11698485083332

# Part 2 - Number of Ether Transactions Per Day

In [116]:
ETH_DAILY_TRANSACTIONS

,Date(UTC),UnixTimeStamp,Value
0,7/30/2015,1438214400,8893
1,7/31/2015,1438300800,0
2,8/1/2015,1438387200,0
3,8/2/2015,1438473600,0
4,8/3/2015,1438560000,0
...,...,...,...
2363,1/17/2022,1642377600,1198719
2364,1/18/2022,1642464000,1169526
2365,1/19/2022,1642550400,1223143
2366,1/20/2022,1642636800,1190018


In [117]:
eth_tx_formatted = ETH_DAILY_TRANSACTIONS \
    .astype({ "Date(UTC)": np.datetime64 }) \
    .set_index("Date(UTC)")\
    ["Value"]

eth_tx_formatted

Date(UTC)
2015-07-30       8893
2015-07-31          0
2015-08-01          0
2015-08-02          0
2015-08-03          0
               ...   
2022-01-17    1198719
2022-01-18    1169526
2022-01-19    1223143
2022-01-20    1190018
2022-01-21    1189898
Name: Value, Length: 2368, dtype: int64

In [133]:
# This function queries the number of transactions per day for any specific UNIX timestamp

def get_daily_transactions(lookup, epoch):
    tab = lookup.loc[:np.datetime64(epoch, "s")]
    if not tab.empty:
        # Get last index you found
        return tab.iloc[-1]
    else:
        return lookup.iloc[0]

In [134]:
get_daily_transactions(eth_tx_formatted, 1641837499)

1233195

# Part 3 - Ether Energy Consumption Per Day

In [136]:
def get_energy_kwh_per_trans(currency, epoch):
    total = query_twh_per_yr(currency["energy"], epoch)
    ntx = get_daily_transactions(currency["tx_cnt"], epoch)
    
    return ((total / 365.25) / ntx) * 1e9

In [137]:
ETHEREUM = {
    "energy": eth_energy_formatted,
    "tx_cnt": eth_tx_formatted
}
get_energy_kwh_per_trans(ETHEREUM, 1641837499)

235.59329262763737

# Part 4 - Bitcoin

In [138]:
# Getting Bitcoin Data
# Sources:
# https://digiconomist.net/bitcoin-energy-consumption
# https://www.blockchain.com/charts/n-transactions

BTC_ENERGY = pd.read_csv("bitcoin-energy-consumpti.csv")
BTC_TRANSACTIONS = pd.read_csv("btc-transactions.csv")

In [142]:
BTC_ENERGY

,DateTime,Estimated TWh per Year,Minimum TWh per Year
0,2017-02-10 00:00:00,9.585659,3.331026
1,2017-02-11 00:00:00,9.547335,3.316227
2,2017-02-12 00:00:00,9.578774,3.198969
3,2017-02-13 00:00:00,9.476506,3.045178
4,2017-02-14 00:00:00,9.544807,3.150062
...,...,...,...
1801,2022-01-17 00:00:00,204.495517,55.540599
1802,2022-01-18 00:00:00,204.495517,56.820048
1803,2022-01-19 00:00:00,204.495517,58.127091
1804,2022-01-20 00:00:00,204.495517,58.694202


In [147]:
btc_transactions_processed = BTC_TRANSACTIONS.astype({ "Timestamp": np.datetime64 }).set_index("Timestamp")["n-transactions"]
btc_energy_processed = BTC_ENERGY.astype({ "DateTime": np.datetime64 }).set_index("DateTime")["Estimated TWh per Year"]

BITCOIN = {
    "energy": btc_energy_processed,
    "tx_cnt": btc_transactions_processed
}

In [148]:
get_energy_kwh_per_trans(BITCOIN, 1641837499)

2664.2724797255783